In [17]:
from __future__ import print_function

import numpy as np
from numpy import pi, sin, cos

import cv2

import random
import threading as thd
import time

# built-in modules
from time import clock

# local modules
#import common
import copy

import cv2
#import video
from matplotlib import pyplot as plt

def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    
    cv2.polylines(img, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(img, (x1, y1), 1, (0, 255, 0), -1)
    return img

def analyze_flow(flow,flow_len):
    h, w   = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang    = (np.arctan2(fy,fx)/np.pi + 1) * 180
    v      = np.sqrt(fx*fx+fy*fy)
    fv     = (v >= flow_len).astype('int')
    xProj  = (fv.sum(0).flatten()>=flow_len).nonzero()
    yProj  = (fv.sum(1).flatten()>=flow_len).nonzero()
    
    if xProj[0].size and yProj[0].size:
        pt1    = (xProj[0][0], yProj[0][0])
        pt2    = (xProj[0][-1]+1,yProj[0][-1]+1)
        vidx   = fv[pt1[1]:pt2[1],pt1[0]:pt2[0]].flatten().nonzero()
        if vidx[0].size:
            ang    = ang[pt1[1]:pt2[1],pt1[0]:pt2[0]].flatten()[vidx]
            hist   = cv2.calcHist([ang], [0], None, [8], [0,360])
            ret    = True
        else:
            hist   = []
            ret    = False
    else:
        pt1    = (0,0)
        pt2    = (0,0)
        hist   = []
        ret    = False

    return ret,hist,pt1,pt2

# 子執行緒的工作函數
def fn(img):
    digit = random.randint(1, 4)
    
    return digit
    #t = threading.Thread(target = tm)
    # 執行該子執行緒
    #t.start()
    
    #time.sleep(1)
    #flagTm = True


if __name__ == '__main__':
    
    cap  = cv2.VideoCapture(0)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT));
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH));
    fps    = int(cap.get(cv2.CAP_PROP_FPS));
    
    score = 0
    digit = 1
    
    shot_idx = 0
    while cap.isOpened():
        ret, img = cap.read()
        
        cv2.putText(img,'SCORE:',(300,50),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),3)
        cv2.putText(img,str(score),(520,50),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),3)
        
        
        if ret == True:        
            if shot_idx == 0:
                pregray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                imgf    = img
                res     = False
            else:
                gray    = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                flow    = cv2.calcOpticalFlowFarneback(pregray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                pregray = gray
                res,hist,pt1,pt2 = analyze_flow(flow,6)
                imgf    = draw_flow(img, flow)
                if res and  pt2[1]-pt1[1] >= img.shape[0]/6 and pt2[0]-pt1[0]>=img.shape[1]/5:
                    cv2.rectangle(imgf,pt1,pt2,(0,0,255),2);
                    cv2.line(imgf,(pt1[0]*2//3+pt2[0]//3, pt1[1]),(pt1[0]*2//3+pt2[0]//3, pt2[1]),(0,255,255),2);
                    cv2.line(imgf,(pt1[0]//3+pt2[0]*2//3, pt1[1]),(pt1[0]//3+pt2[0]*2//3, pt2[1]),(0,255,255),2);
                    cv2.line(imgf,(pt1[0],pt1[1]*2//3+pt2[1]//3),(pt2[0],pt1[1]*2//3+pt2[1]//3),(0,255,255),2);
                    cv2.line(imgf,(pt1[0],pt1[1]//3+pt2[1]*2//3),(pt2[0],pt1[1]//3+pt2[1]*2//3),(0,255,255),2);
                    mv = np.max(hist)
                    h  = imgf.shape[0]

                    #for i in range(0,8):
                    #    cv2.rectangle(imgf,(i*10,h-hist[i]/mv*100),(i*10+10,h),(0,255,255),-2)
                    cv2.rectangle(imgf,pt1,pt2,(0,255,255),2);

                    up    = hist[1]+hist[2]
                    down  = hist[5]+hist[6]
                    right = hist[3]+hist[4]
                    left  = hist[0]+hist[7] 
                    if left/2 > right and left/2 > down and left/2 > up:
                        cv2.putText(imgf,'Right',(0,50),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),3)
                        if digit == 1:
                            score += 1
                            digit = fn(img)
                    elif right > left and right > down and right > up:
                        cv2.putText(imgf,'Left',(0,50),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),3)
                        if digit == 2:
                            score += 1
                            digit = fn(img)
                    elif down/2 > left and down/2 > right and down/2 > up:
                        cv2.putText(imgf,'Down',(0,50),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),3)
                        if digit == 3:
                            score += 1
                            digit = fn(img)
                    elif up/2 > left and up/2 > right and up/2 > down:
                        cv2.putText(imgf,'Up',(0,50),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),3)
                        if digit == 4:
                            score += 1
                            digit = fn(img)
                            
                    if digit == 1:
                        cv2.putText(img,'RIGHT',(300,100),cv2.FONT_HERSHEY_PLAIN,4,(0,0,255),3)
                    elif digit == 2:
                        cv2.putText(img,'LEFT',(300,100),cv2.FONT_HERSHEY_PLAIN,4,(0,0,255),3)
                    elif digit == 3:
                        cv2.putText(img,'DOWN',(300,100),cv2.FONT_HERSHEY_PLAIN,4,(0,0,255),3)
                    elif digit == 4:
                        cv2.putText(img,'DOWN',(300,100),cv2.FONT_HERSHEY_PLAIN,4,(0,0,255),3)
                        
                    #Right: 1, Left: 2, Down: 3, Up: 4   

            cv2.imshow('video',imgf);
            cv2.waitKey(1)

            shot_idx = 1
        else:
            break        
        ch = cv2.waitKey(1)
        if ch == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
